In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

from skimage.feature import hog, local_binary_pattern, canny
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# ==========================
# Step 1: Load CIFAR-10 Dataset Online
# ==========================
print("📥 Downloading CIFAR-10 dataset...")
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Flatten labels
y_train = y_train.flatten()
y_test = y_test.flatten()

print("✅ CIFAR-10 dataset loaded successfully!")

# ==========================
# Step 2: Preprocess Data
# ==========================
def rgb2gray(images):
    """Convert RGB images to grayscale."""
    return np.dot(images[..., :3], [0.2989, 0.5870, 0.1140])

def preprocess_data(X_train, X_test):
    """Convert images to grayscale, normalize, and flatten."""
    X_train_gray = rgb2gray(X_train) / 255.0
    X_test_gray = rgb2gray(X_test) / 255.0

    X_train_flat = X_train_gray.reshape(X_train_gray.shape[0], -1)
    X_test_flat = X_test_gray.reshape(X_test_gray.shape[0], -1)

    scaler = StandardScaler()
    X_train_flat = scaler.fit_transform(X_train_flat)
    X_test_flat = scaler.transform(X_test_flat)

    return X_train_gray, X_test_gray, X_train_flat, X_test_flat

print("⚙️ Preprocessing images...")
X_train_gray, X_test_gray, X_train_flat, X_test_flat = preprocess_data(X_train, X_test)
print("✅ Data preprocessing complete!")

# ==========================
# Step 3: Feature Extraction
# ==========================
def extract_hog_features(images):
    """Extract HOG features from grayscale images."""
    return np.array([hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False) for img in images])

def extract_lbp_features(images, P=8, R=1):
    """Extract Local Binary Pattern (LBP) features."""
    return np.array([local_binary_pattern(img, P, R, method='uniform').flatten() for img in images])

def extract_edge_features(images):
    """Extract edge features using Canny edge detection."""
    return np.array([canny(img).flatten() for img in images])

def extract_vgg_features(X_train, X_test):
    """Extract deep learning-based features using VGG16."""
    vgg_model = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

    X_train_vgg = vgg_model.predict(preprocess_input(X_train))
    X_test_vgg = vgg_model.predict(preprocess_input(X_test))

    return X_train_vgg.reshape(X_train_vgg.shape[0], -1), X_test_vgg.reshape(X_test_vgg.shape[0], -1)

# Extract features
print("🔍 Extracting HOG features...")
X_train_hog, X_test_hog = extract_hog_features(X_train_gray), extract_hog_features(X_test_gray)

print("🔍 Extracting LBP features...")
X_train_lbp, X_test_lbp = extract_lbp_features(X_train_gray), extract_lbp_features(X_test_gray)

print("🔍 Extracting Edge Detection features...")
X_train_edges, X_test_edges = extract_edge_features(X_train_gray), extract_edge_features(X_test_gray)

print("🔍 Extracting VGG16 features (Deep Learning)...")
X_train_vgg, X_test_vgg = extract_vgg_features(X_train, X_test)

print("✅ Feature extraction complete!")

# ==========================
# Step 4: Train & Evaluate Classifiers
# ==========================
def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name):
    """Train a classifier and evaluate performance."""
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)
    print(f"\n📊 {model_name} Performance:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Classification Report:\n{report}")
    return acc, report

print("🧠 Training and evaluating classifiers...")

# Define classifiers
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
knn_classifier = KNeighborsClassifier(n_neighbors=5)

# Train and evaluate classifiers
acc_hog_rf, report_hog_rf = train_and_evaluate(rf_classifier, X_train_hog, X_test_hog, y_train, y_test, "RandomForest (HOG)")
acc_hog_knn, report_hog_knn = train_and_evaluate(knn_classifier, X_train_hog, X_test_hog, y_train, y_test, "KNN (HOG)")

acc_lbp_rf, report_lbp_rf = train_and_evaluate(rf_classifier, X_train_lbp, X_test_lbp, y_train, y_test, "RandomForest (LBP)")
acc_lbp_knn, report_lbp_knn = train_and_evaluate(knn_classifier, X_train_lbp, X_test_lbp, y_train, y_test, "KNN (LBP)")

acc_edges_rf, report_edges_rf = train_and_evaluate(rf_classifier, X_train_edges, X_test_edges, y_train, y_test, "RandomForest (Edge Detection)")
acc_edges_knn, report_edges_knn = train_and_evaluate(knn_classifier, X_train_edges, X_test_edges, y_train, y_test, "KNN (Edge Detection)")

acc_vgg_rf, report_vgg_rf = train_and_evaluate(rf_classifier, X_train_vgg, X_test_vgg, y_train, y_test, "RandomForest (VGG16)")
acc_vgg_knn, report_vgg_knn = train_and_evaluate(knn_classifier, X_train_vgg, X_test_vgg, y_train, y_test, "KNN (VGG16)")

# ==========================
# Step 5: Display Results
# ==========================
print("\n📊 Final Results:")
print(f"HOG (RandomForest) Accuracy: {acc_hog_rf:.4f}")
print(f"HOG (KNN) Accuracy: {acc_hog_knn:.4f}")

print(f"LBP (RandomForest) Accuracy: {acc_lbp_rf:.4f}")
print(f"LBP (KNN) Accuracy: {acc_lbp_knn:.4f}")

print(f"Edge Detection (RandomForest) Accuracy: {acc_edges_rf:.4f}")
print(f"Edge Detection (KNN) Accuracy: {acc_edges_knn:.4f}")

print(f"VGG16 (RandomForest) Accuracy: {acc_vgg_rf:.4f}")
print(f"VGG16 (KNN) Accuracy: {acc_vgg_knn:.4f}")


📥 Downloading CIFAR-10 dataset...
✅ CIFAR-10 dataset loaded successfully!
⚙️ Preprocessing images...
✅ Data preprocessing complete!
🔍 Extracting HOG features...
🔍 Extracting LBP features...
🔍 Extracting Edge Detection features...
🔍 Extracting VGG16 features (Deep Learning)...
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
✅ Feature extraction complete!
🧠 Training and evaluating classifiers...

📊 RandomForest (HOG) Performance:
Accuracy: 0.5017
Classification Report:
              precision    recall  f1-score   support

           0     0.5940    0.5970    0.5955      1000
           1     0.5593    0.6650    0.6076      1000
           2     0.4562    0.3490    0.3955      1000
           3     0.3570    0.2660    0.3049      1000
           4     0.4081    0.4150    0.4115      1000
           5     0.3850    0.4420    0.4115      1000
           6     0.4933    0.6300    0.5534      1000
           7     0.5878    0.5020    0.5415      1000
    